In [ ]:
from itertools import compress
import pandas as pd
from operator import itemgetter
from itertools import groupby
import json
from bert_serving.client import BertClient
import os.path
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.autograd import Variable  
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image  
import ftplib
import time
import threading

#1.加载训练文件
with open("/data/fjs/data/math/math.json",'r') as load_f:
    allItems = json.load(load_f)#list中条目是dict
    
#2.探索性数据分析EDA
#2.1 习题难度分布
diffs={}#统计难度类别
for item in allItems:
    diff = item['qdiff'] #获取习题难度
    diffs[diff]=diffs.get(diff,0)+1 #统计每个元素出现的个数。
#{0: 9, 1: 5434, 2: 198597, 3: 482016, 4: 212266, 5: 1678}
df = pd.DataFrame(diffs,index=[0])
df.plot(kind='bar')
plt.show()
#2.2 习题概念分布
cons={}#统计概念类别
for item in allItems:
    con =item['qklg']
    conp = con.split(',')#逗号分隔
    for p in conp:#具体概念点
        cons[p]=cons.get(p,0)+1
df = pd.DataFrame(list(cons.items()))
print (df.shape[0])#知识点数
print (np.mean(df.loc[:,1]))#知识点数的均值
#绘制箱形图
def drawBox(freqs):
    #创建箱形图
    #第一个参数为待绘制的定量数据
    #第二个参数为数据的文字说明
    plt.boxplot([freqs], labels=['Concepts'])
    plt.title('Frequecy Of Concpets')
    plt.show()
drawBox(df.loc[:,1])

#3.习题的文本、概念基于BERT模型Embedding；习题的图像基于Resnet模型提取视觉特征
f = ftplib.FTP("10.21.3.24")  # 实例化FTP对象
f.login("ftpuser", "dm2019")  # 登录
def ftp_download(remotename,localname):
    bufsize = 1024  # 设置缓冲器大小
    fp = open(localname, 'wb')
    f.retrbinary('RETR %s' % remotename, fp.write, bufsize)
    fp.close()
#预训练resnet50模型
transform1 = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
resnet50_feature_extractor = models.resnet50(pretrained = True)
resnet50_feature_extractor.fc = nn.Linear(2048, 2048)
torch.nn.init.eye(resnet50_feature_extractor.fc.weight)
for param in resnet50_feature_extractor.parameters():
    param.requires_grad = False
#定义18个线程提取特征，每个线程提取5万条
def threadFunction(n,items):
    print ("Start Threading %d."%n)
    start = time.time()
    VItems = []#保存列表
    for item in items:
        VItem={}#dict
        VItem['quid']=item['quid']
        VItem['qdiff']=item['qdiff']
        #3.1 文本、概念embedding
        texts=item['qtext']#list
        strtext=''
        for text in texts:
            strtext=strtext+text
        concepts=item['qklg']
        bc = BertClient()#连接BERT Server
        VItem['qtext']=bc.encode([strtext])[0]#list,768长度
        VItem['qklg']=bc.encode([concepts])[0]#list，768长度
        #3.2 图片视觉特征提取
        images=[]
        for imgftppath in item['qimage']:
            imgftppath = imgftppath[17:]#截掉ftp://10.21.3.24/
            imglocalpath = "/data/fjs/data/imgs/"+imgftppath.split("/")[-1]
            ftp_download(imgftppath,imglocalpath)#从ftp下载到本地
            image = Image.open(imglocalpath)#打开本地图片
            images.append(image)
        #多图拼接成一张长图
        if len(images)==0:
            VItem['qimage']=np.zeros(2048)
        else:
            width,height=images[0].size
            for image in images:
                w,h = image.size
                if w>width:width=w
                if h>height: height=h
            longImg =  Image.new(images[0].mode,(width,height*len(images)))
            for i,im in enumerate(images):
                longImg.paste(im,box=(0,i*height))
            #长图向量化    
            imgarr = np.array(longImg)
            if imgarr.shape[2] == 4: #四通道转为三通道
                img1 = longImg.convert("RGB")
            #img = Image.fromarray(img.astype('uint8')).convert('RGB')
            img2 = transform1(img1)
            x = Variable(torch.unsqueeze(img2, dim=0).float(), requires_grad=False)
            y = resnet50_feature_extractor(x)
            y = y.data.numpy()
            VItem['qimage']=y #2048长度
        VItems.append(VItem)
    with open("/data/fjs/data/math/mathV"+str(n)+".json","w") as dump_f:
        json.dump(VItems,dump_f)#保存到json文件
    print ('file %d saved.'%n)
    end = time.time()
    print ("Complete time: %f s" % (end - start))

#将全部90万条记录分成18份提取特征
for i in range(0,len(allItems),50000):
    mThread = threading.Thread(target=threadFunction, args=(i,allItems[i:i+50000]))
    mThread.start()

/usr/local/lib/python3.5/dist-packages/torchvision-0.2.1-py3.5.egg/torchvision/transforms/transforms.py:200: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [34]:
import os.path
import json 
import ftplib
from pathlib import Path
#1.加载训练文件
with open("/data/fjs/data/math/math.json",'r') as load_f:
    allItems = json.load(load_f)#list中条目是dict
    
#count =0
f = ftplib.FTP("10.21.3.24")  # 实例化FTP对象
f.login("ftpuser", "dm2019")  # 登录
def ftp_download(remotename,localname):
    bufsize = 1024  # 设置缓冲器大小
    fp = open(localname, 'wb')
    f.retrbinary('RETR %s' % remotename, fp.write, bufsize)
    fp.close()
#将ftp全部手动在操作系统上下载到本地，然后再调整异常的，减少磁盘IO和网络传输时间
imgs=[]
for item in allItems:
    for img in item['qimage']:
        #if img.find('ftp://10.21.3.24/')==-1:print (img)
        #if img.find('upimages')==-1:print (img)
        ipath = img[img.find('upimages'):]#截取ftp://10.21.3.24/tiku/images/后的目录
        ipath = "/data/fjs/data/mathimgs/"+ ipath
        #if img.find('\\')>=0:img = img[:-1] #去除最后一个字符
        if ipath.find('\\')>=0:ipath = ipath[:-1] #去除最后一个字符
        if not os.path.exists(ipath):#本地不存在，去下载。
            img = img[17:]#截掉ftp://10.21.3.24/
            imgs.append(img)
for img in imgs:
    ipath = img[img.find('upimages'):]
    ipath = "/data/fjs/data/mathimgs/"+ ipath
    ftp_download(img,ipath)#从ftp下载到本地 
    print (ipath)

In [43]:
import json
import pandas as pd
#将list中dict的json文件转化为DataFrame并保存为CSV，可以利用python的矩阵计算优势，而不是采用循环
with open("/data/fjs/data/math/math.json",'r') as load_f:
    allItems = json.load(load_f)#list中条目是dict
csvlist=[]
for item in allItems:
    strtext=''
    for text in item['qtext']: strtext=strtext+text
    strimage=''
    for image in item['qimage']:strimage=strimage+","+image
    tmplist=[item['quid'],item['qdiff'],item['qklg'], strtext, strimage]
    csvlist.append(tmplist)
df = pd.DataFrame(csvlist, columns=['quid', 'qdiff', 'qklg','qtext','qimage']) 
print (df.shape)
df.to_csv('/data/fjs/data/math/math.csv',index=False,sep='|')

(900000, 5)


In [12]:
#去除空置，在多模态学习上更有意义。
import pandas as pd
data = pd.read_csv("/data/fjs/data/math/math.csv",sep='|') #print (data.head())
data=data.dropna(axis=0,how='any')#删除包含空置的行，主要是没有图片。
#data = data.fillna('none')
print (data.shape)
#print (data.head())
data.to_csv('/data/fjs/data/math/mathmin.csv',index=False,sep='|')

(234921, 5)


In [3]:
import pandas as pd
import numpy as np
import json
from bert_serving.client import BertClient
import torch
import torch.nn as nn
from torchvision import models,transforms
from torch.autograd import Variable
from PIL import Image
import time

#1.预训练resnet50模型构建
transform1 = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
resnet50_feature_extractor = models.resnet50(pretrained = True)
resnet50_feature_extractor.fc = nn.Linear(2048, 2048)
torch.nn.init.eye(resnet50_feature_extractor.fc.weight)
for param in resnet50_feature_extractor.parameters():
    param.requires_grad = False
    
#2.连接BertService
bc = BertClient()#连接BERT Server

#3.多图拼接长图并向量化函数定义
def fn_ImageToVec(x):
    images=[]
    for img in x.split(",")[1:]:#ftp图片用逗号隔开
        try:
            if img.find('\\'): img=img.replace('\\','')
            ipath = img[img.find('upimages'):]#截取ftp://10.21.3.24/tiku/images/后的目录
            ipath = "/data/fjs/data/mathimgs/"+ ipath
            image = Image.open(ipath)#打开本地图片
            images.append(image)
        except OSError:pass
        continue
    if len(images)==0:return np.zeros((1,2048))
    else:
        #多图拼接成一张长图
        width,height=images[0].size
        for image in images:
            w,h = image.size
            if w>width:width=w
            if h>height: height=h
        longImg =  Image.new(images[0].mode,(width,height*len(images)))
        for i,im in enumerate(images):
            longImg.paste(im,box=(0,i*height))
        #长图向量化    
        imgarr = np.array(longImg)
        if imgarr.shape[2] == 4: #四通道转为三通道
            longImg = longImg.convert("RGB")
        longImg = transform1(longImg)
        x = Variable(torch.unsqueeze(longImg, dim=0).float(), requires_grad=False)
        y = resnet50_feature_extractor(x)
        return y.data.numpy() #2048长度

start = time.time()    
#4.加载训练文件
data = pd.read_csv("/data/fjs/data/math/mathmin.csv",sep='|') 
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#data = data.head()
#5.1概念向量化
vecklg = bc.encode(data['qklg'].tolist())
data = pd.concat([data,pd.DataFrame(vecklg)],axis=1)#768维
data=data.drop(['qklg'], axis=1)
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#5.2文本向量化
vectxt = data['qtext'].apply(lambda x: bc.encode([x])[0])
data = pd.concat([data,pd.DataFrame(vectxt.tolist())],axis=1)#768维
data=data.drop(['qtext'], axis=1)
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#5.3 图像向量化
npvimg=data['qimage'].apply(lambda x: fn_ImageToVec(x)[0])
data = pd.concat([data,pd.DataFrame(npvimg.tolist())],axis=1)#2048维
data=data.drop(['qimage'], axis=1)
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#6.保存向量化后的文件
data.to_csv("/data/fjs/data/math/mathV.csv",index=False,sep='|')
end = time.time()
print ("Complete time: %f s" % (end - start))

/usr/local/lib/python3.5/dist-packages/torchvision-0.2.1-py3.5.egg/torchvision/transforms/transforms.py:200: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.eye is now deprecated in favor of nn.init.eye_.


234921 rows and 5 cols
5 rows and 772 cols


/home/fjs/.local/lib/python3.5/site-packages/bert_serving/client/__init__.py:277: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


5 rows and 1539 cols
5 rows and 3586 cols
Complete time: 6.398251 s


In [45]:
import pandas as pd
import numpy as np
import json
from bert_serving.client import BertClient
import torch
import torch.nn as nn
from torchvision import models,transforms
from torch.autograd import Variable
from PIL import Image
import time

#1.预训练resnet50模型构建
transform1 = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
resnet50_feature_extractor = models.resnet50(pretrained = True)
resnet50_feature_extractor.fc = nn.Linear(2048, 2048)
torch.nn.init.eye(resnet50_feature_extractor.fc.weight)
for param in resnet50_feature_extractor.parameters():
    param.requires_grad = False
    
#2.连接BertService
bc = BertClient()#连接BERT Server

#3.多图拼接长图并向量化函数定义
def fn_ImageToVec(x):
    images=[]
    for img in x.split(",")[1:]:#ftp图片用逗号隔开
        if img.find('\\'): img=img.replace('\\','')
        ipath = img[img.find('upimages'):]#截取ftp://10.21.3.24/tiku/images/后的目录
        ipath = "/data/fjs/data/mathimgs/"+ ipath
        image = Image.open(ipath)#打开本地图片
        images.append(image)

    if len(images)==0:return np.zeros((1,2048))
    else:
        #多图拼接成一张长图
        width,height=images[0].size
        for image in images:
            w,h = image.size
            if w>width:width=w
            if h>height: height=h
        longImg =  Image.new(images[0].mode,(width,height*len(images)))
        for i,im in enumerate(images):
            longImg.paste(im,box=(0,i*height))
        #长图向量化    
        imgarr = np.array(longImg)
        if imgarr.shape[2] == 4: #四通道转为三通道
            longImg = longImg.convert("RGB")
        longImg = transform1(longImg)
        x = Variable(torch.unsqueeze(longImg, dim=0).float(), requires_grad=False)
        y = resnet50_feature_extractor(x)
        return y.data.numpy() #2048长度

#4.加载训练文件
dataIn = pd.read_csv("/data/fjs/data/math/mathmin.csv",sep='|') 
print ("%d rows and %d cols"%(dataIn.shape[0],dataIn.shape[1]))

for i in range(0,dataIn.shape[0],100):
    try:
        start = time.time() 
        data=dataIn[i:i+100]
        print (data.shape)
        #5.1概念向量化
        vecklg = bc.encode(data['qklg'].tolist())
        data = pd.concat([data,pd.DataFrame(vecklg)],axis=1)#768维
        data=data.drop(['qklg'], axis=1)
        #5.2文本向量化
        vectxt = data['qtext'].apply(lambda x: bc.encode([str(x)])[0])
        data = pd.concat([data,pd.DataFrame(vectxt.tolist())],axis=1)#768维
        data=data.drop(['qtext'], axis=1)
        #5.3 图像向量化
        npvimg=data['qimage'].apply(lambda x: fn_ImageToVec(str(x))[0])
        data = pd.concat([data,pd.DataFrame(npvimg.tolist())],axis=1)#2048维
        data=data.drop(['qimage'], axis=1)
        #6.保存向量化后的文件
        print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
        data.to_csv("/data/fjs/data/math/max/mathv"+str(i)+".csv",index=False,sep='|')
        end = time.time()
        print ("file %d Complete time: %f s" % (i,(end - start)))
    except Exception:pass
    continue

/usr/local/lib/python3.5/dist-packages/torchvision-0.2.1-py3.5.egg/torchvision/transforms/transforms.py:200: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.eye is now deprecated in favor of nn.init.eye_.


234921 rows and 5 cols
(100, 5)


/home/fjs/.local/lib/python3.5/site-packages/bert_serving/client/__init__.py:277: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


100 rows and 3586 cols
file 0 Complete time: 37.075272 s
(100, 5)
200 rows and 3586 cols
file 100 Complete time: 35.891687 s
(100, 5)
300 rows and 3586 cols
file 200 Complete time: 36.220275 s
(100, 5)
400 rows and 3586 cols
file 300 Complete time: 36.650671 s
(100, 5)
400 rows and 3586 cols
file 400 Complete time: 37.623131 s
(100, 5)
400 rows and 3586 cols
file 500 Complete time: 36.507936 s
(100, 5)
400 rows and 3586 cols
file 600 Complete time: 36.622813 s
(100, 5)
400 rows and 3586 cols
file 700 Complete time: 37.806572 s
(100, 5)
400 rows and 3586 cols
file 800 Complete time: 38.063151 s
(100, 5)
400 rows and 3586 cols
file 900 Complete time: 37.046798 s
(100, 5)
400 rows and 3586 cols
file 1000 Complete time: 37.863056 s
(100, 5)
400 rows and 3586 cols
file 1100 Complete time: 37.595042 s
(100, 5)
400 rows and 3586 cols
file 1200 Complete time: 37.377395 s
(100, 5)
400 rows and 3586 cols
file 1300 Complete time: 37.677139 s
(100, 5)
400 rows and 3586 cols
file 1400 Complete time

400 rows and 3586 cols
file 11900 Complete time: 38.362356 s
(100, 5)
400 rows and 3586 cols
file 12000 Complete time: 38.038853 s
(100, 5)
400 rows and 3586 cols
file 12100 Complete time: 37.127180 s
(100, 5)
400 rows and 3586 cols
file 12200 Complete time: 38.469965 s
(100, 5)
400 rows and 3586 cols
file 12300 Complete time: 38.399041 s
(100, 5)
400 rows and 3586 cols
file 12400 Complete time: 37.201553 s
(100, 5)
400 rows and 3586 cols
file 12500 Complete time: 37.412860 s
(100, 5)
400 rows and 3586 cols
file 12600 Complete time: 38.898150 s
(100, 5)
400 rows and 3586 cols
file 12700 Complete time: 38.171540 s
(100, 5)
400 rows and 3586 cols
file 12800 Complete time: 37.668962 s
(100, 5)
400 rows and 3586 cols
file 12900 Complete time: 38.832823 s
(100, 5)
400 rows and 3586 cols
file 13000 Complete time: 37.919673 s
(100, 5)
400 rows and 3586 cols
file 13100 Complete time: 37.064829 s
(100, 5)
400 rows and 3586 cols
file 13200 Complete time: 37.607529 s
(100, 5)
400 rows and 3586 co

...

400 rows and 3586 cols
file 187400 Complete time: 37.986668 s
(100, 5)
400 rows and 3586 cols
file 187500 Complete time: 39.382112 s
(100, 5)
400 rows and 3586 cols
file 187600 Complete time: 38.615852 s
(100, 5)
400 rows and 3586 cols
file 187700 Complete time: 39.120854 s
(100, 5)
400 rows and 3586 cols
file 187800 Complete time: 39.025224 s
(100, 5)
400 rows and 3586 cols
file 187900 Complete time: 38.537030 s
(100, 5)
400 rows and 3586 cols
file 188000 Complete time: 38.693903 s
(100, 5)
400 rows and 3586 cols
file 188100 Complete time: 38.208907 s
(100, 5)
400 rows and 3586 cols
file 188200 Complete time: 37.654221 s
(100, 5)
400 rows and 3586 cols
file 188300 Complete time: 38.393277 s
(100, 5)
400 rows and 3586 cols
file 188400 Complete time: 38.862350 s
(100, 5)
400 rows and 3586 cols
file 188500 Complete time: 38.547292 s
(100, 5)
400 rows and 3586 cols
file 188600 Complete time: 38.235995 s
(100, 5)
400 rows and 3586 cols
file 188700 Complete time: 38.780002 s
(100, 5)
400 ro

400 rows and 3586 cols
file 199000 Complete time: 38.702112 s
(100, 5)
400 rows and 3586 cols
file 199100 Complete time: 39.285722 s
(100, 5)
400 rows and 3586 cols
file 199200 Complete time: 39.412596 s
(100, 5)
400 rows and 3586 cols
file 199300 Complete time: 39.000638 s
(100, 5)
400 rows and 3586 cols
file 199400 Complete time: 38.676094 s
(100, 5)
400 rows and 3586 cols
file 199500 Complete time: 38.025641 s
(100, 5)
400 rows and 3586 cols
file 199600 Complete time: 38.574508 s
(100, 5)
400 rows and 3586 cols
file 199700 Complete time: 38.609142 s
(100, 5)
400 rows and 3586 cols
file 199800 Complete time: 37.789576 s
(100, 5)
400 rows and 3586 cols
file 199900 Complete time: 38.752322 s
(100, 5)
400 rows and 3586 cols
file 200000 Complete time: 38.647509 s
(100, 5)
400 rows and 3586 cols
file 200100 Complete time: 38.955267 s
(100, 5)
400 rows and 3586 cols
file 200200 Complete time: 38.377105 s
(100, 5)
400 rows and 3586 cols
file 200300 Complete time: 39.121481 s
(100, 5)
400 ro

400 rows and 3586 cols
file 210600 Complete time: 37.162365 s
(100, 5)
400 rows and 3586 cols
file 210700 Complete time: 37.636213 s
(100, 5)
400 rows and 3586 cols
file 210800 Complete time: 37.799084 s
(100, 5)
400 rows and 3586 cols
file 210900 Complete time: 38.351600 s
(100, 5)
400 rows and 3586 cols
file 211000 Complete time: 37.976937 s
(100, 5)
400 rows and 3586 cols
file 211100 Complete time: 38.078650 s
(100, 5)
400 rows and 3586 cols
file 211200 Complete time: 37.539019 s
(100, 5)
400 rows and 3586 cols
file 211300 Complete time: 37.580058 s
(100, 5)
400 rows and 3586 cols
file 211400 Complete time: 38.356629 s
(100, 5)
400 rows and 3586 cols
file 211500 Complete time: 37.497446 s
(100, 5)
400 rows and 3586 cols
file 211600 Complete time: 38.497648 s
(100, 5)
400 rows and 3586 cols
file 211700 Complete time: 38.096209 s
(100, 5)
400 rows and 3586 cols
file 211800 Complete time: 38.294777 s
(100, 5)
400 rows and 3586 cols
file 211900 Complete time: 38.496109 s
(100, 5)
400 ro

400 rows and 3586 cols
file 222200 Complete time: 39.048132 s
(100, 5)
400 rows and 3586 cols
file 222300 Complete time: 39.990425 s
(100, 5)
400 rows and 3586 cols
file 222400 Complete time: 38.922221 s
(100, 5)
400 rows and 3586 cols
file 222500 Complete time: 38.703436 s
(100, 5)
400 rows and 3586 cols
file 222600 Complete time: 39.454183 s
(100, 5)
400 rows and 3586 cols
file 222700 Complete time: 39.289584 s
(100, 5)
400 rows and 3586 cols
file 222800 Complete time: 39.571431 s
(100, 5)
400 rows and 3586 cols
file 222900 Complete time: 39.755972 s
(100, 5)
400 rows and 3586 cols
file 223000 Complete time: 38.744359 s
(100, 5)
400 rows and 3586 cols
file 223100 Complete time: 39.180846 s
(100, 5)
400 rows and 3586 cols
file 223200 Complete time: 39.494209 s
(100, 5)
400 rows and 3586 cols
file 223300 Complete time: 38.613689 s
(100, 5)
400 rows and 3586 cols
file 223400 Complete time: 39.431972 s
(100, 5)
400 rows and 3586 cols
file 223500 Complete time: 39.388295 s
(100, 5)
400 ro

400 rows and 3586 cols
file 233900 Complete time: 39.701821 s
(100, 5)
400 rows and 3586 cols
file 234000 Complete time: 39.714111 s
(100, 5)
400 rows and 3586 cols
file 234100 Complete time: 39.419843 s
(100, 5)
400 rows and 3586 cols
file 234200 Complete time: 39.775583 s
(100, 5)
400 rows and 3586 cols
file 234300 Complete time: 39.641631 s
(100, 5)
400 rows and 3586 cols
file 234400 Complete time: 39.473780 s
(100, 5)
400 rows and 3586 cols
file 234500 Complete time: 39.209243 s
(100, 5)
400 rows and 3586 cols
file 234600 Complete time: 39.052407 s
(100, 5)
400 rows and 3586 cols
file 234700 Complete time: 39.991096 s
(100, 5)
400 rows and 3586 cols
file 234800 Complete time: 39.677312 s
(21, 5)
84 rows and 3586 cols
file 234900 Complete time: 8.198071 s


In [ ]:
import pandas as pd
import os
#读取小文件汇总成大文件，并去重
dataout = pd.read_csv("/data/fjs/data/math/max/mathv0.csv",sep='|') 
for i in range(0,234900,100):
    ipath = "/data/fjs/data/math/max/mathv"+str(i)+".csv"
    if os.path.exists(ipath):#判断文件是否存在
        dataIn = pd.read_csv(ipath,sep='|') 
        dataout = pd.concat([dataout,dataIn],axis=0)#行拼接
        print ("file %d Complete" % i)
dataout = dataout.drop_duplicates()#去重
print ("%d rows and %d cols"%(dataout.shape[0],dataout.shape[1]))
dataout.to_csv("/data/fjs/data/math/mathV.csv",index=False,sep='|')

file 0 Complete
file 100 Complete
file 200 Complete


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


file 300 Complete
file 400 Complete
file 500 Complete
file 600 Complete
file 700 Complete
file 800 Complete
file 900 Complete
file 1000 Complete
file 1100 Complete
file 1200 Complete
file 1300 Complete
file 1400 Complete
file 1500 Complete
file 1600 Complete
file 1700 Complete
file 1800 Complete
file 1900 Complete
file 2000 Complete
file 2100 Complete
file 2200 Complete
file 2300 Complete
file 2400 Complete
file 2500 Complete
file 2600 Complete
file 2700 Complete
file 2800 Complete
file 2900 Complete
file 3000 Complete
file 3100 Complete
file 3200 Complete
file 3300 Complete
file 3400 Complete
file 3500 Complete
file 3600 Complete
file 3700 Complete
file 3800 Complete
file 3900 Complete
file 4000 Complete
file 4100 Complete
file 4200 Complete
file 4300 Complete
file 4400 Complete
file 4500 Complete
file 4600 Complete
file 4700 Complete
file 4800 Complete
file 4900 Complete
file 5000 Complete
file 5100 Complete
file 5200 Complete
file 5300 Complete
file 5400 Complete
file 5500 Complete
